In [1]:
import json
import urllib.request
from datetime import datetime
from datetime import timedelta
import re

In [5]:
def collectClip(channel, lim, Clientid, File):
    url = "https://api.twitch.tv/kraken/clips/top?channel=" + channel + "&limit=" + str(lim)
    req = urllib.request.Request(url, headers = {"Client-ID": Clientid, "Accept" : "application/vnd.twitchtv.v5+json"})
    u = urllib.request.urlopen(req)
    c = u.read().decode('utf-8')
    js = json.loads(c)

    return js
    #collectChat(js, lim, Clientid, File)

def collectChat(j, limit, clientId, f):
    for num in range(int(limit)):
        id = j['clips'][num]['vod']['id']
        offset = j['clips'][num]['vod']['offset']
        duration = j['clips'][num]['duration']

        cursor = ""
        count = 0

        while(1):
            try:
                url2 = ""
                if count == 0:
                    url2 = "https://api.twitch.tv/kraken/videos/" + str(id) + "/comments?content_offset_seconds=" + str(offset)
                else:
                    url2 = "https://api.twitch.tv/kraken/videos/" + str(id) + "/comments?cursor=" + str(cursor)
                req2 = urllib.request.Request(url2, headers = {"Client-ID": clientId, "Accept" : "application/vnd.twitchtv.v5+json"})
                u2 = urllib.request.urlopen(req2)
                c2 = u2.read().decode('utf-8')
                j2 = json.loads(c2)
                endCount = 0
                try:
                    for number, com in enumerate(j2['comments']):

                        dateString = j2['comments'][number]['created_at']
                        if "." in dateString:
                            dateString = re.sub(r".[0-9]+Z","Z", dateString)
                        date = datetime.strptime(dateString, "%Y-%m-%dT%H:%M:%SZ")

                        if (int(offset) + int(duration)) < int(j2['comments'][number]['content_offset_seconds']):
                            endCount = 1
                            break
                        else:
                            f.write(str(j['clips'][num]['title']) + "\t" +
                                    str(j['clips'][num]['game']) + "\t" +
                                    str(j['clips'][num]['views']) + "\t" +
                                    str(j['clips'][num]['duration']) + "\t" +
                                    str(j['clips'][num]['vod']['url']) + "\t" +
                                    str(date + timedelta(hours=9)) + "\t" +
                                    str(j2['comments'][number]['commenter']['name']) + "\t" +
                                    str(j2['comments'][number]['message']['body']) + "\n")

                except Exception as e:
                    print(e)

                if endCount == 1:
                    break

                if j2['_next']:
                    cursor = j2['_next']

                count = count + 1

            except Exception as e:
                print(e)

In [6]:
if __name__ == "__main__":
    file = open("clipchat.txt", "w", encoding="utf-8")
    Channel = "hanryang1125"
    Limit = 1
    ClientId = "7lwx13o4czugl09409d7nntm1epfwt" # Client id 추가
    js = collectClip(Channel, Limit, ClientId, file)
    file.close()

In [7]:
js

# id = js['clips'][num]['vod']['id']
# offset = js['clips'][num]['vod']['offset']
# duration = js['clips'][num]['duration']

{'clips': [{'slug': 'SleepyInspiringPuffinGivePLZ',
   'tracking_id': '659780404',
   'url': 'https://clips.twitch.tv/SleepyInspiringPuffinGivePLZ?tt_medium=clips_api&tt_content=url',
   'embed_url': 'https://clips.twitch.tv/embed?clip=SleepyInspiringPuffinGivePLZ&tt_medium=clips_api&tt_content=embed',
   'embed_html': "<iframe src='https://clips.twitch.tv/embed?clip=SleepyInspiringPuffinGivePLZ&tt_medium=clips_api&tt_content=embed' width='640' height='360' frameborder='0' scrolling='no' allowfullscreen='true'></iframe>",
   'broadcaster': {'id': '103825127',
    'name': 'hanryang1125',
    'display_name': '풍월량',
    'channel_url': 'https://www.twitch.tv/hanryang1125',
    'logo': 'https://static-cdn.jtvnw.net/jtv_user_pictures/hanryang1125-profile_image-58261d78af47d249-150x150.jpeg'},
   'curator': {'id': '137897233',
    'name': 'symgathy12',
    'display_name': '키레이토',
    'channel_url': 'https://www.twitch.tv/symgathy12',
    'logo': 'https://static-cdn.jtvnw.net/jtv_user_pictures